In [1]:
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd
import urllib.request


In [2]:
#df1 = pd.read_csv("small-2oq-c1r.csv", error_bad_lines=False, low_memory=False)
#print(df1)

#df2 = df1.set_index('categories', drop=False)
#df3 = df2.loc["Apparels>Women>Western Wear>Shirts, Tops & Tunics>Tops"]
#df3.to_csv('tops_listings_small.csv', index=False)

In [3]:
df3 = pd.read_csv("tops_listings_small.csv")
df3.shape


C:\Users\DELL\Anaconda3\envs\dog-project\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(333162, 32)

In [4]:
def getImageUrls(prod):
    url_str = prod.imageUrlStr
    img_urls = url_str.split(';')
    return img_urls

In [5]:


def getImagesFromUrls(urls):
    img=[]
    for url in urls:
        req = urllib.request.urlopen(url)
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img.append(cv2.imdecode(arr, -1)) # 'Load it as it is'
        print(np.shape(img))
        
    return img


In [6]:
urls = getImageUrls(df3.iloc[0])

print("total images in this listing are :", len(urls))
urls[0]

image = getImagesFromUrls(urls)
print(len(image))
for i in range (len(image)):
    print(image[i].shape)


total images in this listing are : 4
(1, 1498, 667, 3)
(2,)
(3,)
(4,)
4
(1498, 667, 3)
(200, 89, 3)
(400, 178, 3)
(800, 356, 3)


In [7]:
#cv2.imshow('image', image[1])
#cv2.waitKey(2)

In [8]:
i=0
df3.iloc[i]
#df3.iloc[53].description

productId                                               TOPE9ABBZU3HZRHN
title                  Citrine Casual Short Sleeve Printed Women's Pi...
description            This beautiful printed modal top from Citrine ...
imageUrlStr            http://img.fkcdn.com/image/top/r/h/n/1-1-wwtpw...
mrp                                                                 1099
sellingPrice                                                         329
specialPrice                                                         329
productUrl             http://dl.flipkart.com/dl/citrine-casual-short...
categories             Apparels>Women>Western Wear>Shirts, Tops & Tun...
productBrand                                                     Citrine
productFamily          TOPE9ABBBTJYDSQE,TOPE9ABBHJ8HGGGK,TOPE9ABBPDAN...
inStock                                                            False
codAvailable                                                        True
offers                                             

In [9]:
def areImagesSameForProducts(prod1, prod2):
    if(prod1.imageUrlStr == prod2.imageUrlStr):
        return True
    return False

In [12]:
#areImagesSameForProducts(df3.iloc[13], df3.iloc[14])



In [16]:
from scipy import spatial
from gensim import models

model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

index2word_set = set(model.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

C:\Users\DELL\Anaconda3\envs\dog-project\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [27]:
s1_afv = avg_feature_vector(df3.iloc[298].description, model=model, num_features=300, index2word_set=index2word_set)
s2_afv = avg_feature_vector(df3.iloc[299].description, model=model, num_features=300, index2word_set=index2word_set)
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)

0.537812385646


In [29]:
def isDuplicate(prod1, prod2):
    if(prod1.productId == prod2.productId):
        return True
    if(prod1.productUrl == prod2.productUrl):
        return True
    
    #if(prod1.title == prod2.title):
    #    if(prod1.description == prod2.description):
    #        return True
        
    if(prod1.imageUrlStr == prod2.imageUrlStr):
        return True

    return False


In [28]:
#df3.shape[0]
count = 0
for i in range(0, (int)(df3.shape[0]/1000.0)):
    for j in range(i+1, (int)(df3.shape[0]/1000.0)):
        if(isDuplicate(df3.iloc[i], df3.iloc[j])):
            count +=1
            print(count, i, j)
            #print("duplicate products:", i,j, df3.iloc[i].productUrl, df3.iloc[j].productUrl)

1 0 1
2 5 6
3 10 11
4 13 14
5 22 68
6 22 69
7 24 25
8 27 28
9 29 30
10 39 40
11 48 70
12 48 71
13 51 52
14 51 53
15 51 54
16 52 53
17 52 54
18 53 54
19 62 63
20 62 64
21 63 64
22 68 69
23 70 71
24 75 76
25 79 80
26 87 88
27 87 296
28 88 296
29 93 94
30 95 96
31 100 101
32 102 308
33 103 104
34 109 110
35 113 114
36 119 120
37 123 124
38 125 126
39 133 134
40 135 136
41 141 143
42 142 144
43 142 145
44 144 145
45 147 149
46 147 150
47 147 151
48 149 150
49 149 151
50 150 151
51 154 157
52 155 156
53 163 303
54 167 168
55 173 174
56 173 175
57 174 175
58 183 184
59 183 185
60 184 185
61 186 322
62 189 190
63 189 191
64 189 309
65 190 191
66 190 309
67 191 309
68 194 195
69 196 304
70 199 200
71 199 203
72 200 203
73 205 208
74 206 207
75 210 211
76 210 212
77 211 212
78 213 214
79 213 310
80 213 329
81 214 310
82 214 329
83 215 216
84 215 332
85 216 332
86 217 218
87 224 225
88 228 229
89 228 231
90 228 232
91 228 233
92 228 234
93 229 231
94 229 232
95 229 233
96 229 234
97 231 232
98 2